In [45]:
#text mining

import pandas as pd
import nltk
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
import re

In [47]:
df=pd.read_csv('spam_ham_dataset.csv')

In [93]:
email=(df['text'][0])# 1st email

In [95]:
email=email.lower() # lower case all characteres

In [97]:
email=re.sub(r'\d+','',email) #remove digits

In [99]:
email.translate(str.maketrans('','',string.punctuation)) #remove punctuations

'subject enron methanol  meter   \r\nthis is a follow up to the note i gave you on monday        preliminary\r\nflow data provided by daren  \r\nplease override pop  s daily volume  presently zero  to reflect daily\r\nactivity you can obtain from gas control \r\nthis change is needed asap for economics purposes '

In [101]:
email=email.replace('subject','') #remove subject

In [103]:
email=email.strip() #removing extra space from the begining and the ending of the email

In [105]:
from nltk.stem import PorterStemmer,WordNetLemmatizer
st=PorterStemmer()
#le=WordNetLemmatizer()

In [107]:
email

": enron methanol ; meter # : \r\nthis is a follow up to the note i gave you on monday ,  /  /  { preliminary\r\nflow data provided by daren } .\r\nplease override pop ' s daily volume { presently zero } to reflect daily\r\nactivity you can obtain from gas control .\r\nthis change is needed asap for economics purposes ."

In [109]:
email=' '.join([st.stem(word) for word in email.split()]) #stammed each word

In [111]:
 email

": enron methanol ; meter # : thi is a follow up to the note i gave you on monday , / / { preliminari flow data provid by daren } . pleas overrid pop ' s daili volum { present zero } to reflect daili activ you can obtain from ga control . thi chang is need asap for econom purpos ."

In [113]:
#create function here
def clean_emails(email):
    
    email=email.lower() # lower case all characteres
    email=re.sub(r'\d+','',email) #remove digits
    email.translate(str.maketrans('','',string.punctuation)) #remove punctuations
    email=email.replace('subject','') #remove subject
    email=email.strip() #removing extra space from the begining and the ending of the email
    st=PorterStemmer()
    email=' '.join([st.stem(word) for word in email.split()]) #stammed each word
    return email


In [155]:
df['clean text']=df['text'].apply(clean_emails)

In [153]:
df

,label,text,label_num,clean text
0,ham,Subject: enron methanol ; meter # : 988291\r\n...,0,: enron methanol ; meter # : thi is a follow u...
1,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0,": hpl nom for januari , ( see attach file : hp..."
2,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0,": neon retreat ho ho ho , we ' re around to th..."
3,spam,"Subject: photoshop , windows , office . cheap ...",1,": photoshop , window , offic . cheap . main tr..."
4,ham,Subject: re : indian springs\r\nthis deal is t...,0,: re : indian spring thi deal is to book the t...
...,...,...,...,...
5166,ham,Subject: put the 10 on the ft\r\nthe transport...,0,: put the on the ft the transport volum decrea...
5167,ham,Subject: 3 / 4 / 2000 and following noms\r\nhp...,0,: / / and follow nom hpl can ' t take the extr...
5168,ham,Subject: calpine daily gas nomination\r\n>\r\n...,0,": calpin daili ga nomin > > juli , as i mentio..."
5169,ham,Subject: industrial worksheets for august 2000...,0,: industri worksheet for august activ attach a...


In [119]:
#tf-idf
#Term Frequency - Inverse Document Frequency

from sklearn.feature_extraction.text import TfidfVectorizer

In [125]:
tfidf=TfidfVectorizer(stop_words='english')
vectorized_text=tfidf.fit_transform(['i like apples','i like oranges','i like apples and oranges'])

In [127]:
pd.DataFrame(vectorized_text.toarray(),columns=tfidf.get_feature_names_out())

,apples,like,oranges
0,0.789807,0.613356,0.000000
1,0.000000,0.613356,0.789807
2,0.619805,0.481334,0.619805


In [163]:
X=tfidf.fit_transform(df['clean text'])
y=df['label_num']

In [165]:
df.shape

(5171, 4)

In [167]:
X

<5171x37763 sparse matrix of type '<class 'numpy.float64'>'
	with 292348 stored elements in Compressed Sparse Row format>

In [169]:
from sklearn.model_selection import train_test_split

In [171]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

In [173]:
from sklearn.naive_bayes import MultinomialNB

In [175]:
nb=MultinomialNB()

In [177]:
nb.fit(X_train,y_train)

MultinomialNB()

In [179]:
y_pred=nb.predict(X_test)

In [181]:
from sklearn.metrics import classification_report

In [183]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.90      1.00      0.95       717
           1       1.00      0.74      0.85       318

    accuracy                           0.92      1035
   macro avg       0.95      0.87      0.90      1035
weighted avg       0.93      0.92      0.92      1035



In [185]:
import joblib
joblib.dump(nb,'Predicting spam email model.pkl')

['Predicting spam email model.pkl']

In [189]:
def predict_email(email):
    cleaned=clean_emails(email)
    vector=tfidf.transform([cleaned])
    pred=nb.predict(vector)
    if pred==1:
        print('This seems like a spam email')
    else:
        print('This seems like a genuine email')

In [191]:
example_text='Congratulation!!! you have won a free IPhone! Click here to claim it now'
predict_email(example_text)

This seems like a spam email
